In [23]:
import pandas as pd
import openai 

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct


# Read the sampled Amazon inventory data

In [2]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl",lines=True)

In [3]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Electronics,2 Pack-iPhone Earbuds Wired Lightning Headphon...,3.4,598,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],WASABI MANGO,"[Electronics, Headphones, Earbuds & Accessorie...",{'Product Dimensions': '23.62 x 19.69 x 27.56 ...,B0B1ZVC7GJ,NaN,NaN,NaN
1,Computers,"Mini PC 16GB DDR4 256GB M.2 SSD,Quad-Core 2.7G...",4.3,450,[【Meet to Sufficient Memory Storage】This Mini ...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],OUVISLITE,"[Electronics, Computers & Accessories, Compute...","{'Screen Resolution': '3840 x 2160', 'Max Scre...",B0B1HNV2V9,NaN,NaN,NaN
2,Computers,Samsers Foldable Bluetooth Keyboard with Touch...,4.4,308,[【Full-size Folding Wireless Keyboard】 Samsers...,[],48.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Love this keyboard!', 'url': 'http...",Samsers,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '13.5 x 4.5 x 0.4 inche...,B0C2Q8BDTX,NaN,NaN,NaN
3,Computers,"Rolling Laptop Bag Women with Wheels, Rolling ...",4.5,152,[MOBILE OFFICE: EMPSIGN rolling bag with lapto...,[rolling laptop bag],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Elegant Laptop Tote Bag for Women ...,Ytonet,"[Electronics, Computers & Accessories, Laptop ...","{'Brand': 'Ytonet', 'Item model number': '1332...",B092Z9CTNK,NaN,NaN,NaN
4,All Electronics,"Wireless Mouse, 2.4G Silent Mouse with USB Rec...",4.6,677,[Plug & Play Super Easy to Use- Just plug and ...,[],10.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],MagoFeliz,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '4.69 x 2.6 x 0.01 inch...,B0C77L1G7V,NaN,NaN,NaN


In [4]:
list(df_items.features.items())[1]

(1,
 ['【Meet to Sufficient Memory Storage】This Mini Computer equiped with 16GB DDR4 and 256GB SSD. While ensuring cost-effectiveness, the larger capacity brings you a better multitasking experience. Newbie friendly Storage upgrade:1)2.5 inch SATA SSD/HDD up to 2TB(not include). 2)replace the M.2 SATA HDD. Expand your applications, files, video or audio freely.',
  '【J4125 Mini PC with Windows 11 Pro】Celeron Gemini Lake J4125 Quad-Core processor with base frequency of 2.0 GHz and up to 2.7 GHz in burst. Let the minis to run multiple applications, Web pages, full HD video or light game smoothly. Windows 11 preinstalled and automatically activated after connecting to the Internet, plug and play!',
  '【Enjoy Immersive Visual Experience】The desktop computer has 2 HDMI ports and 1 VGA port for triple screen output, supporting three-screen simultaneous display or different display. The combination with HD Graphics 600 (up to 750MHz) makes work more efficient, 4K@60Hz HD brings you a visual fe

In [5]:
list(df_items.images.items())

[(0,
  [{'thumb': 'https://m.media-amazon.com/images/I/31eziY1O3EL._AC_US40_.jpg',
    'large': 'https://m.media-amazon.com/images/I/31eziY1O3EL._AC_.jpg',
    'variant': 'MAIN',
    'hi_res': 'https://m.media-amazon.com/images/I/51tDCwwyEgL._AC_SL1417_.jpg'},
   {'thumb': 'https://m.media-amazon.com/images/I/51J0Iti28iL._AC_US40_.jpg',
    'large': 'https://m.media-amazon.com/images/I/51J0Iti28iL._AC_.jpg',
    'variant': 'PT01',
    'hi_res': 'https://m.media-amazon.com/images/I/71voGQrX7nL._AC_SL1276_.jpg'},
   {'thumb': 'https://m.media-amazon.com/images/I/419vSzdw3pL._AC_US40_.jpg',
    'large': 'https://m.media-amazon.com/images/I/419vSzdw3pL._AC_.jpg',
    'variant': 'PT02',
    'hi_res': 'https://m.media-amazon.com/images/I/61Iub5dt2SL._AC_SL1500_.jpg'},
   {'thumb': 'https://m.media-amazon.com/images/I/4111pWhpUIL._AC_US40_.jpg',
    'large': 'https://m.media-amazon.com/images/I/4111pWhpUIL._AC_.jpg',
    'variant': 'PT03',
    'hi_res': 'https://m.media-amazon.com/images/I/61

# Process title and features

In [6]:
def preprocess_description(row):
    return f"{row["title"]}{' '.join(row["features"])}"

In [7]:
def extract_images(row):
    return row["images"][0].get("large","")

In [8]:
df_items["description"] = df_items.apply(preprocess_description,axis=1)
df_items["images"] = df_items.apply(extract_images, axis=1)   

In [9]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Electronics,2 Pack-iPhone Earbuds Wired Lightning Headphon...,3.4,598,[],2 Pack-iPhone Earbuds Wired Lightning Headphon...,NaN,https://m.media-amazon.com/images/I/31eziY1O3E...,[],WASABI MANGO,"[Electronics, Headphones, Earbuds & Accessorie...",{'Product Dimensions': '23.62 x 19.69 x 27.56 ...,B0B1ZVC7GJ,NaN,NaN,NaN
1,Computers,"Mini PC 16GB DDR4 256GB M.2 SSD,Quad-Core 2.7G...",4.3,450,[【Meet to Sufficient Memory Storage】This Mini ...,"Mini PC 16GB DDR4 256GB M.2 SSD,Quad-Core 2.7G...",NaN,https://m.media-amazon.com/images/I/31d48aRFMM...,[],OUVISLITE,"[Electronics, Computers & Accessories, Compute...","{'Screen Resolution': '3840 x 2160', 'Max Scre...",B0B1HNV2V9,NaN,NaN,NaN
2,Computers,Samsers Foldable Bluetooth Keyboard with Touch...,4.4,308,[【Full-size Folding Wireless Keyboard】 Samsers...,Samsers Foldable Bluetooth Keyboard with Touch...,48.99,https://m.media-amazon.com/images/I/51bW+ihg8x...,"[{'title': 'Love this keyboard!', 'url': 'http...",Samsers,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '13.5 x 4.5 x 0.4 inche...,B0C2Q8BDTX,NaN,NaN,NaN
3,Computers,"Rolling Laptop Bag Women with Wheels, Rolling ...",4.5,152,[MOBILE OFFICE: EMPSIGN rolling bag with lapto...,"Rolling Laptop Bag Women with Wheels, Rolling ...",NaN,https://m.media-amazon.com/images/I/01RmK+J4pJ...,[{'title': 'Elegant Laptop Tote Bag for Women ...,Ytonet,"[Electronics, Computers & Accessories, Laptop ...","{'Brand': 'Ytonet', 'Item model number': '1332...",B092Z9CTNK,NaN,NaN,NaN
4,All Electronics,"Wireless Mouse, 2.4G Silent Mouse with USB Rec...",4.6,677,[Plug & Play Super Easy to Use- Just plug and ...,"Wireless Mouse, 2.4G Silent Mouse with USB Rec...",10.99,https://m.media-amazon.com/images/I/31cV2lTLOG...,[],MagoFeliz,"[Electronics, Computers & Accessories, Compute...",{'Product Dimensions': '4.69 x 2.6 x 0.01 inch...,B0C77L1G7V,NaN,NaN,NaN


In [10]:
list(df_items["description"].items())[0]

(0,
 '2 Pack-iPhone Earbuds Wired Lightning Headphone【Apple MFi Certified】in-Ear Headset Stereo Noise Canceling with Built-in Microphone & Volume Control Compatible with iPhone 13/12/11/SE/X/XR/8/7-All iOS')

In [11]:
list(df_items.images.items())[0]

(0, 'https://m.media-amazon.com/images/I/31eziY1O3EL._AC_.jpg')

# sample 50 itesm from the dataset

In [12]:


df_sample = df_items.sample(50, random_state=42)

In [13]:
data_to_embed = df_sample[["description","images","rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [14]:
data_to_embed


[{'description': 'YKZ 3 in 1 Magnetic C to C Cable 4 Pack, Magnet Type C to Type C Charging Cable, [3.3FT+6.0FT] PD 60W Fast Charger Nylon Braided Cord for Type C/MicroUSB and i-Product【3 in 1 USB C to USB C cable】- This cable has 3 different magnet connectors, suitable for Type-c, Micro-USB devices and i-Product. (NOTE: This is the USB-C output cable, you may need a USB C wall charger to charge your device.) 【PD 60W Fast Charge】- YKZ type C to type C Cable supports Power Delivery fast charge up to 20V/3A (max), 15V/3A, 12V/3A, 9V/3A, 5V/3A at 60W with USB-C PD adapters such as PD 18W/30W/45W/60W. (Note: Charging speeds also depends on your device and charger. ) 【Magnetic Charging Cable】- Magnetic charging cable has a strong magnet that can attract the phone without falling off. You can easily connect with one hand while driving or doing other things. Besides, Magnetic tips can be used as a dust plug that reducing abrasion during inserting or plugging out the cable, greatly prolong the

# Define the embeddings function


In [15]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from .env file in the project root
env_path = Path(__file__).parent.parent.parent / '.env' if '__file__' in globals() else Path('../../.env')
load_dotenv(dotenv_path=env_path)

# Set the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Verify the key is loaded
print(f"API Key loaded: {openai.api_key[:10]}..." if openai.api_key else "API Key NOT loaded!")

API Key loaded: sk-proj-sh...


In [16]:
response = openai.embeddings.create(
    input="Random text",
    model="text-embedding-3-small",
)

In [18]:
len(response.data[0].embedding)


1536

In [19]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

# Create Qdrant collection


In [24]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [25]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536,distance=Distance.COSINE)

)

True

# Embed data

Test

In [26]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("Test text"),
    payload={
    "text":"Random text",
    "model":"text-embedding-3-small",
    },
)

In [27]:
pointstruct

PointStruct(id=0, vector=[-0.020057253539562225, 0.006970119196921587, 0.037700485438108444, -0.040323127061128616, -0.01916317082941532, -0.0343029722571373, 0.0005480913096107543, -0.02425944246351719, 0.03871377930045128, 0.0015953787369653583, 0.030667034909129143, 0.012792832218110561, -0.010863103903830051, 0.011287793517112732, 0.02639034017920494, 0.04303517937660217, -0.046820126473903656, -0.00690678833052516, -0.02174110896885395, 0.05108192190527916, 0.0070856050588190556, 0.013113211840391159, 0.01130269467830658, -0.032544609159231186, -0.00393396383151412, -0.039041608572006226, -0.036955416202545166, 0.004965884145349264, 0.05841339752078056, -0.07462609559297562, 0.031382299959659576, -0.044018667191267014, -0.00025891143013723195, -0.010974864475429058, 0.006005255039781332, 0.03814752772450447, 0.03030940145254135, 0.037640880793333054, 0.010274499654769897, -0.029355714097619057, -0.014916278421878815, -0.010386260226368904, 0.021279167383909225, 0.01814987696707248

In [ ]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload=data,
        )
    )

In [30]:
pointstructs

[PointStruct(id=0, vector=[0.01983162946999073, -0.030867556110024452, -0.033469103276729584, -0.013637471944093704, -0.05896838754415512, -0.0024208836257457733, -0.013183233328163624, 0.018850889056921005, 0.018468914553523064, -0.029938431456685066, 0.02130790427327156, 5.141635483596474e-05, -0.024652749300003052, -0.008687307126820087, 0.030805613845586777, 0.04102597385644913, -0.06223064288496971, -0.017643027007579803, 0.01013261079788208, 0.020461369305849075, 0.003615839872509241, 0.04257451370358467, 0.03427434340119362, -0.011283691972494125, -0.04509347304701805, -0.024900516495108604, 0.02283579669892788, 0.039766497910022736, 0.011934078298509121, 0.016404196619987488, -0.026676174253225327, -0.01883024163544178, -0.02696523629128933, -0.033469103276729584, -0.04069561883807182, 0.013141939416527748, -0.014731773175299168, 0.025292813777923584, 0.029215779155492783, 0.0030996601562947035, 0.010142934508621693, -0.012140550650656223, 0.05079209804534912, -0.01637322455644

# Write embedded data to Qdrant


In [31]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Data Retrival function


In [32]:
def retrieve_data(query, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

# Test Retrieval 

In [33]:
retrieve_data("What kind of tablets can I get?", k=10).points


[ScoredPoint(id=22, version=0, score=0.3613905, payload={'description': 'DUODUOGO 2 in 1 Tablet 10 Inch with Keyboard Mouse Stylus, 128GB Expand 64GB ROM 4GB RAM Android 10.0 Quad-Core HD IPS Screen 8MP Dual Camera GPS FM OTG Bluetooth 4G Dual SIM & WiFi, DGO-P610.1 Inch HD IPS Tablet - This tablet features a 1280*800 HD G+G IPS touch display, wide viewing angle, bright display, more vivid colors and top-notch picture quality. High Performance 2 in 1 Tablet - Equipped with 4GB RAM, 64GB ROM, you can expand the storage up to 128GB using a Micro SD card (not included). Perfect for daily study and entertainment use. Android 10.0 Tablet - An Android 10.0 tablet can watch videos like YouTube and Netflix. Use ZOOM, Skype, etc. to join the meeting. Also chat with friends on Facebook, WhatsApp. LONG BATTERY LIFE - 8000mAh built-in rechargeable battery and low-power CPU enhance your experience, with long runtime and standby time, you can enjoy up to 8 hours. PERFECT GIFT - The DUODUOGO P8 table